# LUNA Lung Segmentation ROI


# Dependency Descriptions
1. **skimage.morphology**: Morphological operations, e.g., opening or skeletonization.
2. **skimage.measure**: Measurement of image properties, e.g., similarity and contours.
3. **sklearn.cluster.KMeans**: clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares
4. **skimage.transform.resize**: Geometric and other transforms, e.g., rotation or the Radon transform.

In [1]:
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
from glob import glob

In [2]:
WORKING_PATH = "../../../../output/build-simple-model/"
FILE_LIST = glob(WORKING_PATH + "images_*.npy")